In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

from peft import get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [ ]:
torch.cuda.is_available()

## Sentimental analysis based on given sentences

In [ ]:
model_checkpoint = "distilbert-base-uncased"

# label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

# classification model from checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

### Load dataset

In [ ]:
dataset = load_dataset("shawhin/imdb-truncated") # optionally: imdb for full dataset
dataset

### Create tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model.resize_token_embeddings(len(tokenizer))

### Define tokenizer function

In [ ]:
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    # tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text, return_tensors="np", truncation=True, max_length=512
    )

    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

### Create data collator for dynamically padding shorter sequences

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=-1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)}

In [ ]:
sample_text_list = ["It was good.", "Not a fan, don't recommand.", "Better than first one.", "Not worth the time.", "This one is a pass"]

print("Non fine-tuned model predictions:")
print("-----------------------------------")

for text in sample_text_list:
    # tokenize
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # get predicted label
    preds = torch.argmax(logits)

    print(f"{text} - {id2label[preds.tolist()]}")

In [ ]:
peft_config = LoraConfig(
    task_type="SEQ_CLS",# sequence classification
    r=4, # intrinsic rank of trainable weight matrix
    lora_alpha=32, # lora LR
    lora_dropout=0.01, # lora dropout
    target_modules=["q_lin"] # target layer for lora -> query linear layer
)

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

### hyperparameters

In [ ]:
lr = 1e-3
batch_size = 4
num_epochs = 10

### training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir=model_checkpoint + "_lora-text-cls",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

### trainer class

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

In [ ]:
model.to("cpu")

print("Fine-tuned model predictions:")
print("-----------------------------------")
for text in sample_text_list:
    # tokenize
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # get predicted label
    preds = torch.max(logits,1).indices

    print(f"{text} - {id2label[preds.tolist()[0]]}")

```
Based on the results from training, we can observe training loss is dropping, validation loss is rising and accuracy has minor improment.
We may assume that overfitting happened here since we directly jumped to LORA fine-tuning.
Instead, perhaps first try out transfer learning of the model to compare the result beofre moving to LORA to fine-tune even furter
```